Imports necessários

In [ ]:
import requests
from io import BytesIO
import zipfile
import pandas as pd

Download e extração

In [ ]:
# URL do arquivo compactado
url_do_arquivo_zip = 'https://portaldatransparencia.gov.br/download-de-dados/viagens/2023'

# Baixando o arquivo compactado
resposta = requests.get(url_do_arquivo_zip)

# Verificando se a solicitação foi bem-sucedida
if resposta.status_code == 200:
    # Lendo o conteúdo do arquivo compactado
    conteudo_zip = BytesIO(resposta.content)

    # Extraindo o conteúdo do arquivo compactado
    with zipfile.ZipFile(conteudo_zip, 'r') as zip_ref:
        # Local da extracao
        zip_ref.extractall('../csv_files')
    print("Conteúdo extraído com sucesso!")
else:
    print(f"Falha ao baixar o arquivo. Código de status: {resposta.status_code}")

Leitura e transformacao dos arquivos .csv em DataFrame

In [ ]:
arquivo_viagem = '../csv_files/2023_Viagem.csv'
arquivo_passagem = '../csv_files/2023_Passagem.csv'
arquivo_trecho = '../csv_files/2023_Trecho.csv'
arquivo_pagamento = '../csv_files/2023_Pagamento.csv'

# Carregando os dados do arquivo CSV em um DataFrame do pandas
df_viagem = pd.read_csv(arquivo_viagem, sep=';', encoding="latin-1")

df_passagem = pd.read_csv(arquivo_passagem, sep=';', encoding="latin-1")

df_trecho = pd.read_csv(arquivo_trecho, sep=';', encoding="latin-1")

df_pagamento = pd.read_csv(arquivo_pagamento, sep=';', encoding="latin-1")


Tratamento dos dados do dataframe Viagem

In [ ]:
# Remover duplicatas mantendo apenas a última ocorrência de ID_PROCESSO_VIAGEM
df_viagem = df_viagem.drop_duplicates(subset='Identificador do processo de viagem', keep='last')

# Removendo colunas que não serão utilizadas
colunas_to_remove = ['Código do órgão superior', 'Nome do órgão superior', 'Código órgão solicitante', 'Nome órgão solicitante', 'Função',
                     'Descrição Função', 'Período - Data de início', 'Período - Data de fim', 'Destinos', 'Valor diárias', 'Valor passagens']
df_viagem.drop(colunas_to_remove, axis=1, inplace=True)

# Tratamento para as colunas com valores monetarios
colunas_to_numeric = ['Valor devolução', 'Valor outros gastos']
for coluna in colunas_to_numeric:
    df_viagem[coluna] = df_viagem[coluna].str.replace(',', '.')
    df_viagem[coluna] = df_viagem[coluna].apply(pd.to_numeric, errors='coerce')

novos_nomes = {'Identificador do processo de viagem': 'ID_PROCESSO_VIAGEM','Número da Proposta (PCDP)': 'NUM_PROPOSTA',
               'Situação': 'SITUACAO','Viagem Urgente':'URGENTE','Justificativa Urgência Viagem': 'JUSTIFICATIVA_URGENCIA',
               'CPF viajante': 'CPF','Nome' : 'NOME','Cargo': 'CARGO','Motivo': 'MOTIVO',
               'Valor devolução': 'VALOR_DEVOLUCAO','Valor outros gastos': 'VALOR_OUTROS'}
df_viagem.rename(columns=novos_nomes, inplace=True)


In [12]:
df_viagem

,ID_PROCESSO_VIAGEM,NUM_PROPOSTA,SITUACAO,URGENTE,JUSTIFICATIVA_URGENCIA,CPF,NOME,CARGO,MOTIVO,VALOR_DEVOLUCAO,VALOR_OUTROS
0,18288418,000007/23-1C,Realizada,SIM,Por necessidade do serviço.,***.621.358-**,ANDRE LUIS COSTA PITANGUEIRA,NaN,Participar da revisão do COE Manual (Contingen...,0.0,1095.35
1,18296348,000070/23,Realizada,SIM,A efetivação do empenho no Siafi e a respectiv...,***.407.547-**,LIDIANY MIRANDA FERRAZ NUNES,AUXILIAR DE BIBLIIOTECA,Participação das aulas de Mestrado Interinstit...,0.0,0.00
2,18302983,000001/23,Realizada,NÃO,Sem informação,***.855.388-**,GREGOR GASPAR,NaN,PARTICIPAR DO CURSO DE PILOTO DE GRIPEN C/D.,0.0,0.00
3,18303291,Informações p,Realizada,NÃO,Informação protegida por sigilo nos termos da ...,ID001000345,Informações protegidas por sigilo,Informações protegidas por sigilo,Informação protegida por sigilo nos termos da ...,0.0,0.00
4,18306758,000002/23,Realizada,NÃO,Sem informação,***.295.391-**,VITOR LUIS MARTINS FARIA,NaN,PARTICIPAR DO CURSO DE PILOTO DE GRIPEN C/D.,0.0,0.00
...,...,...,...,...,...,...,...,...,...,...,...
726802,2023001516,Sem informaçã,Realizada,NÃO,Sem informação,***.249.701-**,LIDIA APARECIDA CURY REISS,TECNICO DO BANCO CENTRAL,Gerência Administrativa de Porto Alegre (ADPAL),0.0,0.00
726803,2023001518,Sem informaçã,Realizada,NÃO,Sem informação,***.244.227-**,RENATO DIAS DE BRITO GOMES,DIRETOR DO BANCO CENTRAL,Participar das Reuniõs do Comef e cumprir outr...,0.0,0.00
726804,2023001523,Sem informaçã,Realizada,NÃO,Sem informação,***.697.505-**,EDSON BROXADO DE FRANCA TEIXEIRA,ANALISTA DO BANCO CENTRAL,"Acompanhar o Diretor Ailton, durante a agenda ...",0.0,0.00
726805,2023001542,Sem informaçã,Realizada,NÃO,Sem informação,***.852.244-**,RAIMUNDO BATISTA FERNANDES DE MELO,TECNICO DO BANCO CENTRAL,Possibilitar o acesso e o fechamento do imóvel...,0.0,0.00


Tratamento dos dados do dataframe Passagens

In [ ]:
# Tratamento para as colunas com valores monetarios
colunas_to_numeric = ["Valor da passagem","Taxa de serviço"]
for coluna in colunas_to_numeric:
    df_passagem[coluna] = df_passagem[coluna].str.replace(',', '.')
    df_passagem[coluna] = df_passagem[coluna].apply(pd.to_numeric, errors='coerce')

# Removendo colunas desnecessárias
colunas_to_remove = ['Número da Proposta (PCDP)']
df_passagem.drop(colunas_to_remove, axis=1, inplace=True)

# Formatando data e hora para formato correto
df_passagem['Data da emissão/compra'] = pd.to_datetime(df_passagem['Data da emissão/compra'], format='%d/%m/%Y', errors='coerce')
df_passagem['Hora da emissão/compra'] = pd.to_datetime(df_passagem['Hora da emissão/compra'], format='%H:%M', errors='coerce').dt.time

novos_nomes = {'Meio de transporte': 'TIPO_PASSAGEM','País - Origem ida': 'PAIS_ORIGEM_IDA',
               'UF - Origem ida': 'UF_ORIGEM_IDA','Cidade - Origem ida': 'CIDADE_ORIGEM_IDA',
               'País - Destino ida': 'PAIS_DESTINO_IDA','UF - Destino ida': 'UF_DESTINO_IDA',
               'Cidade - Destino ida': 'CIDADE_DESTINO_IDA','País - Origem volta': 'PAIS_ORIGEM_VOLTA',
               'UF - Origem volta': 'UF_ORIGEM_VOLTA','Cidade - Origem volta': 'CIDADE_ORIGEM_VOLTA',
               'Pais - Destino volta': 'PAIS_DESTINO_VOLTA','UF - Destino volta': 'UF_DESTINO_VOLTA',
               'Cidade - Destino volta': 'CIDADE_DESTINO_VOLTA','Data da emissão/compra': 'DT_COMPRA',
               'Hora da emissão/compra': 'HR_COMPRA','Valor da passagem': 'VALOR_PASSAGEM',
               'Taxa de serviço': 'VALOR_TAXA_SERVICO','Data da compra': 'DT_COMPRA','Hora da compra': 'HR_COMPRA',
               'Identificador do processo de viagem': 'ID_PROCESSO_VIAGEM'}
df_passagem.rename(columns=novos_nomes, inplace=True)
 

Tratamento de dados do dataframe Trecho

In [ ]:
# Removendo colunas desnecessárias
colunas_to_remove = ['Número da Proposta (PCDP)']
df_trecho.drop(colunas_to_remove, axis=1, inplace=True)

# Tratamento dos valores numericos
colunas_to_numeric = ['Número Diárias']
df_trecho[colunas_to_numeric] = df_trecho[colunas_to_numeric].apply(lambda x: x.str.replace(',', '.'))
df_trecho[colunas_to_numeric] = df_trecho[colunas_to_numeric].apply(pd.to_numeric, errors='coerce')

# Tratamento das datas
colunas_to_date = ['Origem - Data', 'Destino - Data']
df_trecho[colunas_to_date] = df_trecho[colunas_to_date].apply(pd.to_datetime, format='%d/%m/%Y', errors='coerce')

# Conversao para boleano
df_trecho['Missao?'] = df_trecho['Missao?'].apply(lambda x: True if x == 'Sim' else False)

novos_nomes = {'Sequência Trecho': 'SEQUENCIA_VIAGEM','Origem - Data': 'DT_ORIGEM','Origem - País' : 'PAIS_ORIGEM','Origem - UF': 'UF_ORIGEM','Origem - Cidade': 'CIDADE_ORIGEM',
               'Destino - Data': 'DT_DESTINO','Destino - País': 'PAIS_DESTINO','Destino - UF': 'UF_DESTINO','Destino - Cidade': 'CIDADE_DESTINO','Missao?': 'MISSAO',
               'Número Diárias': 'NUM_DIARIAS','Meio de transporte': 'MEIO_TRANSPORTE','Identificador do processo de viagem ': 'ID_PROCESSO_VIAGEM',}
df_trecho.rename(columns=novos_nomes, inplace=True)

Tratamento de dados do dataframe Pagamento

In [ ]:
# Removendo colunas desnecessárias
colunas_to_remove = ['Número da Proposta (PCDP)']
df_pagamento.drop(colunas_to_remove, axis=1, inplace=True)

# Tratamento dos valores numericos
colunas_to_numeric = ['Valor']
df_pagamento[colunas_to_numeric] = df_pagamento[colunas_to_numeric].apply(lambda x: x.str.replace(',', '.'))
df_pagamento[colunas_to_numeric] = df_pagamento[colunas_to_numeric].apply(pd.to_numeric, errors='coerce')

novos_nomes = {'Código do órgão superior': 'COD_ORGAO_SUPERIOR','Nome do órgão superior': 'NOME_ORGAO_SUPERIOR',
               'Codigo do órgão pagador': 'COD_ORGAO_PAGADOR','Nome do órgao pagador': 'NOME_ORGAO_PAGADOR',
               'Código da unidade gestora pagadora': 'COD_UNIDADE_GESTORA_PAGADORA',
               'Nome da unidade gestora pagadora': 'NOME_UNIDADE_GESTORA_PAGADORA','Tipo de pagamento': 
               'TIPO','Valor': 'VALOR','Identificador do processo de viagem': 'ID_PROCESSO_VIAGEM',}
df_pagamento.rename(columns=novos_nomes, inplace=True)

Inserção no banco

In [ ]:
from sqlalchemy import create_engine
import urllib.parse as parse

# Criar a string de conexão
params = parse.quote_plus('')
server_name = 'localhost'
database_name = 'db_viagens_a_servico'
driver_name = 'ODBC+Driver+17+for+SQL+Server'

connection_string = f'mssql+pyodbc://@{server_name}/{database_name}?driver={driver_name}'

# Criar a engine de conexão
engine = create_engine(connection_string)

# Conectar ao banco de dados
conn = engine.connect()

# Escrever os DataFrames para as tabelas SQL
df_viagem.to_sql('TB_VIAGENS', con=engine, if_exists='append', index=False)
df_passagem.to_sql('TB_PASSAGEM', con=engine, if_exists='append', index=False)
df_trecho.to_sql('TB_TRECHO', con=engine, if_exists='append', index=False)
df_pagamento.to_sql('TB_PAGAMENTO', con=engine, if_exists='append',index=False)
# Fechar a conexão
conn.close()

